In [4]:
import librosa
import os
import numpy as np
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
from PIL import Image
from random import shuffle
from tqdm.notebook import tqdm
import pandas as pd
from time import time

import pickle
import csv

from scipy.special import softmax
from sklearn.svm import SVC,LinearSVC
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier,ExtraTreesClassifier
from sklearn import svm,metrics,preprocessing
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing
import torch

#from scipy.misc import imread, imresize

%matplotlib inline

In [1]:
# minimum memory imports
from tqdm.notebook import tqdm
import pandas as pd
from time import time
import os
import numpy as np
import pickle
import csv

from scipy.special import softmax
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC,LinearSVC

In [2]:
DATA_DIR=r'../input/vgaf-features'
IMG_SIZE = 224

classes = ['Positive', 'Neutral', 'Negative']
class_to_idx = {'Positive': 1, 'Neutral': 2, 'Negative': 3}
idx_to_class = {1: 'Positive', 2: 'Neutral', 3: 'Negative'}

In [5]:
train_features_dir = '../input/vgaf-features/enet_b2_8_vgaf_train_part_'
train_features = {}
num_of_parts = 4

for i in range(1, num_of_parts+1):
    with open(train_features_dir + str(i) + '.pickle', 'rb') as handle:
        train_part = pickle.load(handle)
        
    train_features.update(train_part)

In [6]:
val_features_dir = '../input/vgaf-features/enet_b2_8_vgaf_val.pickle'
with open(val_features_dir, 'rb') as handle:
    val_features = pickle.load(handle)

In [7]:
def create_dataset(videoname2features,labelsfile):
    x = []
    y = []
    has_faces=[]
    ind=0
    with open(labelsfile, mode='r') as csvfile:
        labels_reader = csv.reader(csvfile, delimiter=' ')
        for i, row in enumerate(labels_reader):
            if i==0:
                #print('first:',row)
                continue
            # print(row) 
            videoname,label=row[0],int(row[1]) # row[0] - 2_1 videoname, int(row[1]) - 2 label
            # print(videoname,label)
            if videoname in videoname2features.keys(): 
                X_global_features=videoname2features[videoname]
            else:
                X_global_features=[]
            #print(videoname,label,len(X_global_features))
            
            total_features=[]
            for cur_features in X_global_features:
                #print(cur_features.shape)
                if False:
                    total_features.extend(cur_features)
                else:
                    mean_features = (np.mean(cur_features, axis=0))
                    std_features = (np.std(cur_features, axis=0))
                    max_features = (np.max(cur_features, axis=0))
                    min_features = (np.min(cur_features, axis=0))

                    # join several features together
                    #feature = np.concatenate((mean_features, std_features, min_features, max_features), axis=None)                    
                    #feature = np.concatenate((mean_features, std_features, min_features), axis=None)
                    #feature = np.concatenate((mean_features, min_features, max_features), axis=None)
                    feature = np.concatenate((mean_features,std_features), axis=None)
                    #feature = np.concatenate((max_features,mean_features,std_features), axis=None)
                    #feature=max_features

                    total_features.append(feature)
            
            if len(total_features)>0:
                total_features=np.array(total_features)
                mean_features = (np.mean(total_features, axis=0))
                std_features = (np.std(total_features, axis=0))
                max_features = (np.max(total_features, axis=0))
                min_features = (np.min(total_features, axis=0))

                # join several features together
                #feature = np.concatenate((mean_features, std_features, min_features, max_features), axis=None)                    
                #feature = np.concatenate((mean_features, std_features, min_features), axis=None)
                feature = np.concatenate((mean_features,std_features), axis=None)
                #feature = np.concatenate((max_features,std_features), axis=None)
                #feature=std_features
                x.append(feature)
                has_faces.append(1)
            else:
                x.append(np.zeros_like(feature))
                has_faces.append(0)
            y.append(label-1)
    x=np.array(x)
    y=np.array(y)
    has_faces=np.array(has_faces)
    # print(x.shape,y.shape)
    return x,y,has_faces

In [7]:
x_train, y_train, has_faces_train = create_dataset(train_features,os.path.join('../input/vgaf-features/Train_labels.txt'))

In [8]:
x_train.shape

(2661, 5632)

In [9]:
with open('x_train.pickle', 'wb') as handle:
    pickle.dump(x_train, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('y_train.pickle', 'wb') as handle:
    pickle.dump(y_train, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('has_faces_train.pickle', 'wb') as handle:
    pickle.dump(has_faces_train, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [8]:
x_test, y_test, has_faces_test = create_dataset(val_features, os.path.join('../input/vgaf-features/Val_labels.txt'))

In [6]:
x_test.shape

(766, 5632)

In [37]:
with open('x_test.pickle', 'wb') as handle:
    pickle.dump(x_test, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('y_test.pickle', 'wb') as handle:
    pickle.dump(y_test, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('has_faces_test.pickle', 'wb') as handle:
    pickle.dump(has_faces_test, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [3]:
train_test_dir = '../input/vgaf-train-test-processed-dataset/created_dataset_features/'
with open(train_test_dir + 'x_train.pickle', 'rb') as handle:
    x_train = pickle.load(handle)
    
with open(train_test_dir + 'y_train.pickle', 'rb') as handle:
    y_train = pickle.load(handle)
    
with open(train_test_dir + 'has_faces_train.pickle', 'rb') as handle:
    has_faces_train = pickle.load(handle)

In [4]:
with open(train_test_dir + 'x_test.pickle', 'rb') as handle:
    x_test = pickle.load(handle)
    
with open(train_test_dir + 'y_test.pickle', 'rb') as handle:
    y_test = pickle.load(handle)
    
with open(train_test_dir + 'has_faces_test.pickle', 'rb') as handle:
    has_faces_test = pickle.load(handle)

In [9]:
x_train.shape, x_test.shape

((2661, 5632), (766, 5632))

In [10]:
from sklearn import svm,metrics,preprocessing

x_train_norm=preprocessing.normalize(x_train,norm='l2')
x_test_norm=preprocessing.normalize(x_test,norm='l2')

In [11]:
lin_svc = svm.LinearSVC(C=1.1) #0.5 1.1 0.6
   
lin_svc.fit(x_train_norm[has_faces_train==1], y_train[has_faces_train==1])
y_pred = lin_svc.predict(x_test_norm)

print("Accuracy:",metrics.accuracy_score(y_test[has_faces_test==1], y_pred[has_faces_test==1]))
print("Complete accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.6545209176788124
Complete accuracy: 0.6422976501305483


In [12]:
lin_svc = svm.SVC(kernel='rbf',C=1)

lin_svc.fit(x_train_norm[has_faces_train==1], y_train[has_faces_train==1])
y_pred = lin_svc.predict(x_test_norm)

print("Accuracy:",metrics.accuracy_score(y_test[has_faces_test==1], y_pred[has_faces_test==1]))
print("Complete accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.689608636977058
Complete accuracy: 0.6762402088772846


In [13]:
linsvc_proba = []
linsvc_dist = lin_svc.decision_function(x_test_norm)
for i in range(len(linsvc_dist)):
    linsvc_proba.append(softmax(linsvc_dist[i]))

linsvc_proba = np.array(linsvc_proba)


In [ ]:
import pickle
model_name = 'linsvc'
MODEL2EMOTIW_FEATURES=model_name+'_proba_vgaf.pickle' 

print(MODEL2EMOTIW_FEATURES)

with open(MODEL2EMOTIW_FEATURES, 'wb') as handle:
    pickle.dump(linsvc_proba, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [14]:
np.random.seed(42)

rf_clf=RandomForestClassifier(n_estimators=1000,max_depth=12, n_jobs=-1)

if True:    
    rf_clf.fit(x_train_norm[has_faces_train==1], y_train[has_faces_train==1])
    y_pred = rf_clf.predict(x_test_norm)
else:
    clf.fit(x_train[has_faces_train==1], y_train[has_faces_train==1])
    y_pred = clf.predict(x_test)

print("Accuracy:",metrics.accuracy_score(y_test[has_faces_test==1], y_pred[has_faces_test==1]))
print("Complete accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.650472334682861
Complete accuracy: 0.6501305483028721


In [15]:
rf_proba = rf_clf.predict_proba(x_test_norm)

In [ ]:
import pickle
model_name = 'rf'
MODEL2EMOTIW_FEATURES=model_name+'_proba_vgaf.pickle' 

print(MODEL2EMOTIW_FEATURES)

with open(MODEL2EMOTIW_FEATURES, 'wb') as handle:
    pickle.dump(rf_proba, handle, protocol=pickle.HIGHEST_PROTOCOL)

## Audio

In [17]:
!pip install transformers

In [16]:
import torchaudio
import torch
import numpy as np
from transformers import AutoConfig, Wav2Vec2Processor, AutoModelForAudioClassification
import librosa
from tqdm.auto import tqdm

In [17]:
model_name_or_path = "facebook/wav2vec2-base-960h"
pooling_mode = "mean"
processor = Wav2Vec2Processor.from_pretrained(model_name_or_path)
target_sampling_rate = processor.feature_extractor.sampling_rate

Downloading:   0%|          | 0.00/159 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/163 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.56k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/291 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

In [18]:
def preprocess_function_eval(speech_path):   
    speech_array, sampling_rate = librosa.load(speech_path, sr = 16000)
    result = processor(speech_array, sampling_rate=target_sampling_rate, max_length=60000, padding="max_length", truncation='longest_first', return_attention_mask=True)
    len_of_input_data = result['input_values'][0].shape[0]
    padded_array = np.pad(result['input_values'][0], ((0,60000-len_of_input_data)), constant_values=0)
    return padded_array

In [19]:
model = AutoModelForAudioClassification.from_pretrained(
    '../input/w2v2-large-finetuned-vgaf-059', 
    num_labels=3,
    label2id=class_to_idx,
    id2label=idx_to_class,
) # acc 

device = 'cuda:0'
model.to(device)

Wav2Vec2ForSequenceClassification(
  (wav2vec2): Wav2Vec2Model(
    (feature_extractor): Wav2Vec2FeatureEncoder(
      (conv_layers): ModuleList(
        (0): Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation): GELUActivation()
        )
        (1): Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation): GELUActivation()
        )
        (2): Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation): GELUActivation()
        )
        (3): Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,))
          (layer_norm): LayerNorm((512,)

In [20]:
import pandas as pd
val_labels = pd.read_csv('../input/vgaf-features/Val_labels.txt', delimiter = ' ')

classes = ['Positive', 'Neutral', 'Negative']
class_to_idx = {'Positive': 1, 'Neutral': 2, 'Negative': 3}
idx_to_class = {1: 'Positive', 2: 'Neutral', 3: 'Negative'}

In [21]:
import pathlib 

path = pathlib.Path('../input/vgaf-features/Val_VGAF_audio-20220823T161237Z-001/Val_VGAF_audio')

audio_proba = [] 

for vid_name in tqdm(val_labels['Vid_name']): 
    vid_name += '.wav'
    audio_path = path/vid_name
    with torch.no_grad():
        input = torch.from_numpy(np.expand_dims(preprocess_function_eval(audio_path), 0)).to(device)
        output = model(input)
        audio_proba.append(softmax(output[0][0].cpu().numpy()))
audio_proba = np.array(audio_proba)

  0%|          | 0/766 [00:00<?, ?it/s]

In [24]:
with open('../input/w2v2-prob-audio/w2v2_prob_audio_vgaf.pickle', 'rb') as handle:
    audio_proba = pickle.load(handle)

In [20]:
audio_proba

array([[0.04459344, 0.02044857, 0.93495804],
       [0.03691735, 0.01792278, 0.9451598 ],
       [0.86859083, 0.08480235, 0.04660676],
       ...,
       [0.16103035, 0.7461425 , 0.09282707],
       [0.7797291 , 0.04783128, 0.17243962],
       [0.06513524, 0.02386083, 0.91100395]], dtype=float32)

In [ ]:
from sklearn.metrics import accuracy_score

audio_pred = np.argmax(audio_proba, axis=1) + 1
accuracy_score(val_labels['Label'], audio_pred)

In [ ]:
import pickle
model_name = 'w2v2'
MODEL2EMOTIW_FEATURES=model_name+'_prob_audio_vgaf.pickle' 

print(MODEL2EMOTIW_FEATURES)

with open(MODEL2EMOTIW_FEATURES, 'wb') as handle:
    pickle.dump(audio_proba, handle, protocol=pickle.HIGHEST_PROTOCOL)

## Soft Voting

In [31]:
path_to_audio_proba = '../input/w2v2-prob-audio/w2v2_prob_audio_vgaf.pickle'

with open(path_to_audio_proba, "rb") as fh:
    audio_proba = pickle.load(fh)

In [ ]:
path_to_rf_proba = '/content/drive/MyDrive/data/rf_proba_afew.pickle'

with open(path_to_rf_proba, "rb") as fh:
  rf_proba = p.load(fh)

In [ ]:
path_to_of_proba = '/content/drive/MyDrive/data/xgb_of_afew.pickle'

with open(path_to_of_proba, "rb") as fh:
  of_proba = p.load(fh)

In [ ]:
rf_proba

In [ ]:
of_proba

In [32]:
rf_proba.shape, audio_proba.shape, linsvc_proba.shape

((766, 3), (766, 3), (766, 3))

In [33]:
weights = np.arange(0.05, 0.5, 0.01)
small_weights = np.arange(0, 0.1, 0.01)

best_weights = [0, 0, 0] # weight selection on val
best_acc = 0

for r in small_weights:
    for a in weights:
        for l in weights:
            y_pred = []
            for i in range(0, len(linsvc_proba)):
                max_prob = np.argmax(r*rf_proba[i]+a*audio_proba[i]+l*linsvc_proba[i])
                y_pred.append(max_prob)
            if metrics.accuracy_score(y_test, np.array(y_pred)) > best_acc:
                best_acc = metrics.accuracy_score(y_test, np.array(y_pred))
                best_weights[0] = r
                best_weights[1] = a
                best_weights[2] = l
                print(best_acc, best_weights)

0.6684073107049608 [0.0, 0.05, 0.05]
0.6697127937336814 [0.0, 0.05, 0.060000000000000005]
0.6866840731070496 [0.0, 0.05, 0.07]
0.6906005221932114 [0.0, 0.05, 0.08000000000000002]
0.6932114882506527 [0.0, 0.060000000000000005, 0.1]
0.6945169712793734 [0.0, 0.14, 0.23000000000000004]
0.695822454308094 [0.01, 0.1, 0.16000000000000003]
0.6971279373368147 [0.02, 0.08000000000000002, 0.11000000000000001]
0.6984334203655352 [0.03, 0.13, 0.18000000000000005]
0.7010443864229765 [0.04, 0.05, 0.05]
0.7023498694516971 [0.05, 0.05, 0.05]
0.7049608355091384 [0.06, 0.07, 0.060000000000000005]


In [23]:
y_pred = []
mean_best_weights = [0.054, 0.122, 0.144] # the mean of the best weights selected during cross validation

for i in range(0, len(linsvc_proba)):
    max_prob = np.argmax(mean_best_weights[0]*rf_proba[i]+mean_best_weights[1]*audio_proba[i]+mean_best_weights[2]*linsvc_proba[i])
    y_pred.append(max_prob)
    
print(metrics.accuracy_score(y_test, np.array(y_pred)))

0.6932114882506527


In [24]:
y_pred = []
mean_best_weights = [0.054, 0.122, 0.144] # same weights without rf

for i in range(0, len(linsvc_proba)):
    max_prob = np.argmax(mean_best_weights[1]*audio_proba[i]+mean_best_weights[2]*linsvc_proba[i])
    y_pred.append(max_prob)
    
print(metrics.accuracy_score(y_test, np.array(y_pred)))

0.6710182767624021
